<a href="https://colab.research.google.com/github/FIOWB21/Machine-learning/blob/main/Lab_6_Panchenko.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
shlokraval_ppe_dataset_yolov8_path = kagglehub.dataset_download('shlokraval/ppe-dataset-yolov8')

print('Data source import complete.')


100%|██████████| 2.35G/2.35G [00:28<00:00, 88.1MB/s]

Extracting files...


Data source import complete.


In [ ]:
!pip install ultralytics --upgrade --quiet

import torch
from ultralytics import YOLO
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 106.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 93.5 MB/s eta 0:00:00


In [ ]:
# Виведемо кількість тренувальних та валідаційних зображень
print("Train Images:", len(os.listdir(os.path.join('/kaggle/input/ppe-dataset-yolov8/train', 'images'))))
print("Validation Images:", len(os.listdir(os.path.join('/kaggle/input/ppe-dataset-yolov8/valid', 'images'))))


In [ ]:
# Візуалізація прикладу тренувального зображення
sample_img_path = os.path.join('/kaggle/input/ppe-dataset-yolov8','train' , 'images', os.listdir(os.path.join('/kaggle/input/ppe-dataset-yolov8','train', 'images'))[0])
sample_img = cv2.imread(sample_img_path)
sample_img = cv2.cvtColor(sample_img, cv2.COLOR_BGR2RGB)

plt.figure(figsize=(8,8))
plt.imshow(sample_img)
plt.title('Приклад тренувального зображення')
plt.axis('off')
plt.show()

In [ ]:
# Завантаження моделі YOLOv8
model = YOLO('yolov8n.pt')  # Завантажуємо попередньо навчену модель

In [ ]:
# Тренування моделі
results = model.train(
    data='/kaggle/input/ppe-dataset-yolov8/data.yaml',
    epochs=5,
    imgsz=640,
    batch=16,
    device=0,
    name='yolov8_ppe_detection'
)

In [ ]:
# Оцінка моделі
metrics = model.val()  # Оцінка на валідаційному наборі
print("Metrics:", metrics)

In [ ]:
import pandas as pd
# Візуалізація кривих навчання
results_path = '/kaggle/working/runs/detect/yolov8_ppe_detection/results.csv'
if os.path.exists(results_path):
    results_data = pd.read_csv(results_path)

    # Визначимо доступні колонки
    available_columns = results_data.columns
    print("Доступні колонки у results.csv:", available_columns)

    plt.figure(figsize=(12, 8))

    # Втрати (Losses)
    if 'train/box_loss' in available_columns and 'val/box_loss' in available_columns:
        plt.subplot(2, 2, 1)
        plt.plot(results_data['epoch'], results_data['train/box_loss'], label='Train Box Loss')
        plt.plot(results_data['epoch'], results_data['val/box_loss'], label='Val Box Loss')
        plt.title('Box Loss')
        plt.legend()

    if 'train/cls_loss' in available_columns and 'val/cls_loss' in available_columns:
        plt.subplot(2, 2, 2)
        plt.plot(results_data['epoch'], results_data['train/cls_loss'], label='Train Class Loss')
        plt.plot(results_data['epoch'], results_data['val/cls_loss'], label='Val Class Loss')
        plt.title('Class Loss')
        plt.legend()

    # Метрики якості - можуть мати інші назви
    precision_col = next((col for col in available_columns if 'precision' in col.lower()), None)
    recall_col = next((col for col in available_columns if 'recall' in col.lower()), None)

    if precision_col:
        plt.subplot(2, 2, 3)
        plt.plot(results_data['epoch'], results_data[precision_col], label='Precision')
        plt.title('Precision')
        plt.legend()

    if recall_col:
        plt.subplot(2, 2, 4)
        plt.plot(results_data['epoch'], results_data[recall_col], label='Recall')
        plt.title('Recall')
        plt.legend()

    plt.tight_layout()
    plt.show()


In [ ]:
# Тестування моделі на прикладі зображення
test_img_path = os.path.join('/kaggle/input/ppe-dataset-yolov8/valid', 'images', os.listdir(os.path.join('/kaggle/input/ppe-dataset-yolov8/valid', 'images'))[0])
results = model.predict(test_img_path, save=True, conf=0.5)

In [ ]:
# Візуалізація результатів детекції
for result in results:
    result_img = result.plot()  # Генеруємо зображення з bounding boxes
    plt.figure(figsize=(12, 8))
    plt.imshow(result_img)
    plt.title('Результати детекції')
    plt.axis('off')
    plt.show()

    # Виведемо інформацію про детектовані об'єкти
    print("Детектовані об'єкти:")
    for box in result.boxes:
        class_id = int(box.cls)
        class_name = model.names[class_id]
        confidence = float(box.conf)
        print(f"- {class_name}: {confidence:.2f}")

Висновок: Модель YOLOv8 успішно навчена на наборі даних PPE. Криві навчання показують збіжність моделі, зменшення втрат протягом навчання, метрики якості (precision, recall) демонструють ефективність моделі у виявленні об'єктів. Візуалізація результатів детекції підтверджує здатність моделі коректно ідентифікувати засоби захисту на зображеннях.